In [ ]:
import h5py
import numpy as np
import pandas as pd
from pandas import HDFStore
import matplotlib.pyplot as plt
import open3d as o3d
import os
from matplotlib import cm
import cv2 as cv

In [ ]:
file_location = ''
hdf = HDFStore(file_location+'', 'r')

In [ ]:
A = hdf['key2']
A1=A[10:900]
Acut=A1.iloc[:,300:]

In [ ]:
#Data processing
Stopx=(len(Acut.index))*0.3777
x = np.arange(0,Stopx,0.3777)
Stopy=(Acut.shape[1])*0.3925
y = np.arange(0,Stopy,0.3925)
Acut_rename=Acut.copy()
Acut_rename.index=x
Acut_rename.columns=y
Intensity=Acut_rename
Intensity_cut=Intensity.iloc[:,:int(len(Intensity.columns)/2)]

In [ ]:
Ystart=0
Yend=Intensity_cut.shape[0]
Xstart=0
Xend=Intensity_cut.shape[1]
df=Intensity.iloc[Ystart:Yend,Xstart:Xend]
data = df.values

plt.figure()
plt.axis('off') 
plt.margins(0,0)
plt.imshow(data, extent=[Intensity.columns[Xstart], Intensity.columns[Xend-1],Intensity.index[Ystart], Intensity.index[Yend-1]])
plt.savefig('',transparent=True,bbox_inches='tight',pad_inches = -0.1,dpi=600)
plt.show() 

In [ ]:
# Generate some neat n times 3 matrix using a variant of sync function
x = df.index
y = df.columns
mesh_x, mesh_y = np.meshgrid(x, y)
z = np.array(df).transpose()
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(z, -1)

In [ ]:
# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
radius = 1  # Search Radius
max_nn = 9  # Maximum number of points in the neighborhood for estimating normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))

In [ ]:
#Calculate the unit vector in the opposite direction of the laser
points = [[0, 0, 0],[185, 0, 97],[185, 10, 97]]

p0, p1, p2 = points
x0, y0, z0 = p0
x1, y1, z1 = p1
x2, y2, z2 = p2

ux, uy, uz = u = [x1-x0, y1-y0, z1-z0] #first vector
vx, vy, vz = v = [x2-x0, y2-y0, z2-z0] #sec vector

u_cross_v = [uy*vz-uz*vy, uz*vx-ux*vz, ux*vy-uy*vx] #cross product

normal = np.array(u_cross_v)

unit_laser = normal / np.linalg.norm(normal)
unit_laser

In [ ]:
o3d.geometry.PointCloud.orient_normals_to_align_with_direction(pcd, orientation_reference=unit_laser)  
#Set the normal vector in the same direction as the laser inverse

In [ ]:
normals=np.array(pcd.normals)
def cal_angle_of_vector(v0, v1, is_use_deg=True):
    dot_product = np.dot(v0, v1)
    v0_len = np.linalg.norm(v0)
    v1_len = np.linalg.norm(v1)
    try:
        angle_rad = np.arccos(dot_product / (v0_len * v1_len))
    except ZeroDivisionError as error:
        raise ZeroDivisionError("{}".format(error))
 
    if is_use_deg:
        return np.rad2deg(angle_rad)
    return angle_rad

degree=[]
for i in range(len(normals)):
    dd=cal_angle_of_vector(normals[i], unit_laser, is_use_deg=True)
    degree.append(dd)
degree

In [ ]:
degreearray=np.array(degree)
degreearraynew=degreearray.reshape(z.shape)
Finaldegree=pd.DataFrame(degreearraynew.transpose())
Finaldegree.index=x
Finaldegree.columns=y
FinalFinaldegree=Finaldegree.iloc[:,:50]

In [ ]:
def plotcontour(data):
     x = data.index
     y = data.columns
     xs, ys = np.meshgrid(x, y)
     zs = np.array(data).transpose()
     fig=plt.figure(dpi=300)
     ax=fig.add_subplot()
     k=50
     dx=data.index.max()-data.index.min()
     dy=data.columns.max()-data.columns.min()
     dz=data.max().max()-data.min().min()
     norm = cm.colors.Normalize(vmax=abs(zs).max(), vmin=abs(zs).min())
     CD=plt.contour(xs,ys,zs,[0,10,20,30,40,50,60,70,80,90],norm=norm,colors=["red","red","red","red","black","black","black","black","black","black",],linewidths=0.1,alpha=1)
     ax.set_box_aspect(dy/dx)
     plt.axis('off') 
     plt.show()

In [ ]:
def Intensity_Plot(data):
    import plotly.graph_objects as go
    import plotly.express as px
    import plotly.io as pio
    #pio.renderers.default = 'browser'

    fig = px.imshow(np.array(data))#, color_continuous_scale='gray')
    fig.show()

In [ ]:
def show_img(img):
    plt.style.use('dark_background')
    plt.figure(figsize=(10,10)) 
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

In [ ]:
def do_GaussianBlur(gray):
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size), 0)
    return blur_gray

In [ ]:
def do_Canny(blur_gray):
    low_threshold = 100
    high_threshold = 250
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

In [ ]:
def img_processing(img):
    # do something here
    
    print("Gray image:")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # start from BGR -> gray
    #show_img(gray)

    print("GaussianBlur image:")
    blur_img = do_GaussianBlur(gray)
    #show_img(blur_img)
    
    thresh =  cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    
    print("Canny edges:")
    edges = do_Canny(thresh)
    #show_img(edges)
    
    plt.figure(figsize=(15,10)) 
    
    plt.subplot(1, 4, 1)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Origin picture", {'fontsize':20}) 
    
    plt.subplot(1, 4, 2)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(blur_img, cv2.COLOR_BGR2RGB))
    plt.title("GaussianBlur", {'fontsize':20})  
    
    plt.subplot(1, 4, 3)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
    plt.title("Binarization", {'fontsize':20})  

    
    plt.subplot(1, 4, 4)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))
    plt.title("Canny", {'fontsize':20})
    
    plt.show()

In [ ]:
file_name = ""
origin_img = cv2.imread(file_name)
print("origin picture:")
show_img(origin_img)
img_processing(origin_img)

In [ ]:
# The red area indicates the acceptable range
plotcontour(FinalFinaldegree)

In [ ]:
def Augmenteddata(x,y,n,P):
    Type = pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10])
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i)
        Type.loc[loc]=[N1,N2,N3,N4,N5,N6,N7,N8,N9,n]
        #Rotate 90°
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+150)
        Type.loc[loc]=[N7,N4,N1,N8,N5,N2,N9,N6,N3,n]
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+300)
        Type.loc[loc]=[N9,N8,N7,N6,N5,N4,N3,N2,N1,n]
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+450)
        Type.loc[loc]=[N3,N6,N9,N2,N5,N8,N1,N4,N7,n]
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+600)
        Type.loc[loc]=[N3,N2,N1,N6,N5,N4,N9,N8,N7,n]
        #mirror
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+750)
        Type.loc[loc]=[N3,N2,N1,N6,N5,N4,N9,N8,N7,n]
    for i in range(len(y)):
        N1=P.iloc[y[i]-1,x[i]-1]
        N2=P.iloc[y[i]-1,x[i]]
        N3=P.iloc[y[i]-1,x[i]+1]
        N4=P.iloc[y[i],x[i]-1]
        N5=P.iloc[y[i],x[i]]
        N6=P.iloc[y[i],x[i]+1]
        N7=P.iloc[y[i]+1,x[i]-1]
        N8=P.iloc[y[i]+1,x[i]]
        N9=P.iloc[y[i]+1,x[i]+1]
        loc=str(n)+'_'+str(i+900)
        Type.loc[loc]=[N7,N8,N9,N4,N5,N6,N1,N2,N3,n]
        
    return Type

In [ ]:
df1 = pd.read_excel('', sheet_name='Sheet1')
x1=df1.iloc[:,3]
y1=df1.iloc[:,4]

df2 = pd.read_excel('', sheet_name='Sheet2')
x2=df2.iloc[:,3]
y2=df2.iloc[:,4]

df3 = pd.read_excel('', sheet_name='Sheet3')
x3=df3.iloc[:,3]
y3=df3.iloc[:,4]

In [ ]:
type1=Augmenteddata(x1.astype(int),y1.astype(int),1,Profiles)
type2=Augmenteddata(x2.astype(int),y2.astype(int),2,Profiles)
type3=Augmenteddata(x3.astype(int),y3.astype(int),3,Profiles)
frames=[type1,type2,type3]
types = pd.concat(frames)
pd.set_option('display.max_rows', None)

In [ ]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
# load pima indians dataset
# split into input (X) and output (Y) variables
X=types.iloc[:,0:9]
Y=types.iloc[:,9]

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
X=types.iloc[:,0:9]
Y=types.iloc[:,9]

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
# convert integers to dummy variables (one hot encoding)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
# splitting data into training set and test set. If random_state is set to an integer, the split datasets are fixed.
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=0)

In [ ]:
model=Sequential()
model.add(Dense(32, input_dim=9,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [ ]:
model.fit(X_train, Y_train,epochs=100, batch_size=50)

In [ ]:
eval = model.evaluate(X_test, Y_test, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% " % (eval[0], eval[1] * 100) )